TODO: Split data in train, validation, test 

In [104]:
%load_ext autoreload
%autoreload 2
from utils import FusionDataset, TrajectoryPreservingSampler, generate, plot_1d_statistic_over_time
from models import Forward, Posterior, Prior, Decoder
from train import run
import os, pickle
os.environ["KERAS_BACKEND"] = "torch"
import keras

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
# Instantiate models
forward_t = Forward()
forward_tplus1 = Forward()
prior = Prior()
posterior = Posterior()
decoder = Decoder()

# Instantiate optimizer
opt = keras.optimizers.AdamW()

# Get data
dataset = FusionDataset()
dataloader = TrajectoryPreservingSampler(dataset, batch_size=16)

# Run training
run(dataloader, forward_t, forward_tplus1, prior, posterior, decoder, opt, 10)

# Evaluate by generating multiple trajectories from one random starting point
trajectory = next(iter(dataset))
trajectory_hats = []
for i in range(6):
    trajectory_hats.append(keras.ops.concatenate(generate(trajectory, forward_t, prior, decoder)))
    # Save trajectories as figure
    fig = plot_1d_statistic_over_time(trajectory_hats[i].detach().cpu(), 0, "I don't know what this variable is");
    fig.savefig("./resuls/basic0/gen_0")
# Save trajectories as tensors
trajectory_hats = keras.ops.stack(trajectory_hats)
with open("./results/basic0/generated_trajectories.pkl", "wb") as file:
    pickle.dump(trajectory_hats, file)